In [ ]:
import numpy as np
import pandas as pd

def calculate_individual_stock_beta():
    # tickers
    tickers = ['META', 'REGN', 'FI', 'GILD', 'MITK', 'FIS', 'IDCC', 'COUR', 'AMAT', 'KLAC',
               'NICE', 'WFC', 'AMD', 'LLY', 'NVO', 'DELL', 'BP', 'NOK', 'RMBS', 'EL',
               'UIS', 'ILMN', 'GOOG', 'SNPS', 'GRAL', 'UNH', 'GNRC', 'TXN', 'ROK']